# Tracing Nested Calls within Tools

Tools are interfaces an agent can use to interact with any function. Often those functions include additional calls to a retriever, LLM, or other resource you'd like to trace. To ensure the nested calls are all grouped within the same trace, just use the `run_manager`!

The following example shows how to pass callbacks through a tool to a nested chain. The resulting trace looks like the following:

![Trace](./img/trace.png)

## Prerequisites

This example uses OpenAI and LangSmith. Please make sure the dependencies and API keys are configured appropriately before continuing.

In [1]:
# %pip install -U langchain openai

In [2]:
# import os
# os.environ["LANGCHAIN_API_KEY"] = "<your api key>"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["OPENAI_API_KEY"] = "<your openai api key>"

## 1. Define nested tool

Define your tool, taking care to call `run_manager.get_child()` to pass the callback manager through to any functionality you wish to invoke within the tool.

In [3]:
import uuid
from typing import Any, Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import BaseTool
from langchain.schema.runnable import Runnable
from langchain.schema.output_parser import StrOutputParser
import requests
import aiohttp

from pydantic import Field

def _default_summary_chain():
    """An LLM chain that summarizes the input text"""
    return (
        ChatPromptTemplate.from_template(
            "Summarize the following text:\n<TEXT {uid}>\n"
            "{text}"
            "\n</TEXT {uid}>"
        ).partial(uid=lambda: uuid.uuid4())
        | ChatOpenAI(model="gpt-3.5-turbo-16k")
        | StrOutputParser()
    ).with_config(run_name="Summarize Text")

class CustomSummarizer(BaseTool):
    
    name = "summary_tool"
    description = "summarize a website"
    summary_chain: Runnable = Field(default_factory=_default_summary_chain)
        
    def _run(
        self,
        url: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        text = requests.get(url).text
        callbacks = run_manager.get_child() if run_manager else None
        return self.summary_chain.invoke(
            {"text": text},
            {"callbacks": callbacks},
        )

    async def _arun(
        self,
        url: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                text = await response.text()
        callbacks = run_manager.get_child() if run_manager else None
        return await self.summary_chain.ainvoke(
            {"text": text},
            {"callbacks": callbacks},
        )

## 2. Define agent

We will construct a simple agent using runnables and OpenAI functions, following the [Agents overview](https://python.langchain.com/docs/modules/agents/) in the LangChain documentation. The specifics aren't important to this example.

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at summarizing thingws."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [5]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI

# Configure the LLM with access to the appropriate function definitions
llm = ChatOpenAI(temperature=0)
tools = [CustomSummarizer()]
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [6]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
        "chat_history": lambda x: x.get("chat_history") or []
    } 
    | prompt 
    | llm_with_tools 
    | OpenAIFunctionsAgentOutputParser()
).with_config(run_name="Agent")

agent_executor = AgentExecutor(agent=agent, tools=tools)

## 3. Invoke

Now its time to call the agent. All callbacks, including the LangSmith tracer, will be passed to the child runnables.

In [7]:
agent_executor.invoke(
    {"input": f"What's this about? https://blog.langchain.dev/langchain-expression-language/"
    }
)

{'input': "What's this about? https://blog.langchain.dev/langchain-expression-language/",
 'output': 'The website you provided is the LangChain Blog, specifically a blog post about the LangChain Expression Language (LCEL). LCEL is a syntax for creating chains with composition, supporting batch, async, and streaming. The blog post provides information about LCEL, links to resources for learning and using it, and related articles. There is also a newsletter subscription form available on the website.'}

[![Full run trace](./img/full_run.png)](https://smith.langchain.com/public/d0212988-a6d5-4d21-9751-89e77bfa58fa/r)